## k-NN Project 3: Handwritten recognition

We would like to let the machine recognize handwritten digits. The dataset is MNIST comeing from the [MNIST database](https://yann.lecun.com/exdb/mnist/). Now we apply kNN algrotithm to it. 

### Dataset description
Every digit is stored as a $28\times28$ picture. This is a $28\times28$ matrix. Every entry represents a gray value of the corresponding pixel, whose value is from 0 to 255. The label of each matrix is the digit it represents. Note that the dataset provided is already splitted into a training set and a test set.

The dataset can be loaded following the [instruction](https://xiaoxl.github.io/Datasets/contents/mnist.html).


In [ ]:
from datasets import load_dataset
import numpy as np
import itertools

def stream_to_array(streaming, max=None):
    pic_list = []
    label_list =[]
    if max is None:
        generator = streaming
    else:
        generator = itertools.islice(streaming, max)
    for data in generator:
        pic_list.append(np.array(data['image']).reshape(-1))
        label_list.append(data['label'])
    return np.array(pic_list), np.array(label_list)

mnist_train = load_dataset("ylecun/mnist", split='train', streaming=True)
mnist_test = load_dataset("ylecun/mnist", split='test', streaming=True)

X_train, y_train = stream_to_array(mnist_train, max=600)
X_test, y_test = stream_to_array(mnist_test, max=100)

Note that one of the purpose to load the data in streaming mode is that the dataset is big and it is not wise to load everything all together. However this is the only way to train a KNN model since all it does is to memorize everything. In the future with other models we may want to load the image one by one with the streaming mode.

Also due to the issue of large dataset, I only choose the first 600/100 images from the original dataset. The `itertools.islice` is used to only choose the first few items from a generator.


### Apply k-NN
Like the previous two examples, we now try to apply the k-NN algorithm to classify these handwritten digits. Note that the original dataset is huge and the processing time is very slow. However since we only choose 600/100 images, we could still run all our tricks. 


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.base import clone
import matplotlib.pyplot as plt

steps = [('scaler', MinMaxScaler()),
         ('knn', KNeighborsClassifier(n_neighbors=5))]
pipe = Pipeline(steps=steps)
n_list = list(range(1, 11))

cv_score = []
for k in n_list:
    pipe_tmp = clone(pipe)
    pipe_tmp.set_params(knn__n_neighbors=k)
    cv_score.append(cross_val_score(pipe_tmp, X_train, y_train, cv=5).mean())
plt.plot(n_list, cv_score)

In [ ]:
from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(pipe, param_grid=dict(knn__n_neighbors=n_list), cv=5)
gs.fit(X_train, y_train)
gs.best_params_

The best `k=3` for thi degenerated dataset. The corresponding test score is 


In [ ]:
gs.score(X_test, y_test)